In [ ]:
import numpy as np
import plotly.graph_objects as go
import os
import imageio
import cv2
import sys

root_path = '/home/yifengz/workspace_ljh/sim_OKAMI/'

In [ ]:
# read in npy file
depth_img = np.load(root_path + 'depth.npy')
rgb_img = np.load(root_path + 'rgb.npy')

print("shapes: ", depth_img.shape, rgb_img.shape)

# max and min in depth
max_depth = np.max(depth_img)
min_depth = np.min(depth_img)
print("max and min in depth: ", max_depth, min_depth)

extent = 10.60981561453384 
near = 0.0010000000474974513 * extent 
far = 50.0 * extent

depth_img = near / (1 - depth_img * (1 - near / far))

print("after decoding, max and min in depth: ", np.max(depth_img), np.min(depth_img))

In [ ]:
import numpy as np
import plotly.graph_objs as go
import plotly.express as px

# 示例数据生成
# 假设你已经有 depth_image 和 rgb_image
# depth_image = np.random.rand(720, 1280)  # 示例深度图像数据
# rgb_image = np.random.rand(720, 1280, 3)  # 示例 RGB 图像数据

depth_image = depth_img
rgb_image = rgb_img

# 创建 Plotly 图像
fig = px.imshow(rgb_image, binary_string=False)

# 添加深度信息为附加数据
fig.update_traces(customdata=depth_image, hovertemplate='Depth: %{customdata:.8f}<extra></extra>')

# 显示图像
fig.show()

In [ ]:
sys.path.append(root_path)

print(os.getcwd())
os.chdir('/home/yifengz/workspace_ljh/sim_OKAMI/robosuite/')

from scripts_okami.o3d_utils import O3DPointCloud, visualize_o3d_point_cloud
from scripts_okami.plotly_utils import plotly_draw_3d_pcd
import open3d as o3d

In [ ]:
# readin npz file
info = np.load('/home/yifengz/workspace_ljh/sim_OKAMI/pcd_info.npz')

img = info['img'].copy()
# depth = info['depth'].copy()
depth_image = info['depth_image'].copy()
intrinsics = info['camera_intrinsics'].copy()
extrinsics = info['camera_extrinsics'].copy()
mask = info['binary_mask'].copy()

print("shapes: ", img.shape, depth_image.shape, intrinsics.shape, extrinsics.shape, mask.shape)

img = cv2.flip(img, 0)
depth_image = cv2.flip(depth_image, 0)

intrinsics = np.array([
            [909.83630371,   0.        , 651.97015381],
            [  0.        , 909.12280273, 376.37097168],
            [  0.        ,   0.        ,   1.        ],
])

extrinsics = np.eye(4)
extrinsics[:3, 3] = np.array([0.10742, 0.0125, 0.09])
extrinsics[:3, :3] = np.array([
    [ 0.        ,  0.        , 1.        ],
    [-1.        ,  0.        ,  0.        ],
    [ 0.        ,  -1.        ,  0.        ],
])

print("after decoding, max and min in depth_image (parsed object): ", np.max(depth_image), np.min(depth_image))


masked_depth = depth_image * mask
print("for masked depth: ", np.max(masked_depth), np.min(masked_depth))

assert(mask * mask == mask).all()

rgbd_pc = O3DPointCloud(max_points=50000)
rgbd_pc.create_from_rgbd(img, depth_image * 1000, intrinsics, depth_trunc=1.0)
rgbd_pc.transform(extrinsics)
rgbd_pc.preprocess()

pcd_points, pcd_colors = rgbd_pc.get_points(), rgbd_pc.get_colors()
pcd_centers = np.mean(pcd_points, axis=0)

print("pcd centers=", pcd_centers)

plotly_draw_3d_pcd(pcd_points, 
                   pcd_colors,
                   addition_points=np.array([
                       [pcd_centers],
                    #    [target_pos_in_head]
                   ]),
                   marker_size=10)